In [7]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

#Mean Intersection over Union (mIoU)、Dice Coefficient、Precision and Recall:、F1 Score
#input: predict_mask, ground_truth_mask (np.array, shape=(H, W)
#output mIoU, Dice Coefficient, Precision, Recall, F1 Score


# Author: Weijie Zou

import numpy as np
import cv2

def evaluation(predict, ground_truth, num_classes=3):
    IoUs = []
    Dice_scores = []
    Precisions = []
    Recalls = []
    F1_scores = []
    
    for cls in range(num_classes):
        pred_cls = (predict == cls)
        gt_cls = (ground_truth == cls)
        
        intersection = np.logical_and(pred_cls, gt_cls).sum()
        union = np.logical_or(pred_cls, gt_cls).sum()
        IoU = intersection / union if union != 0 else 0
        IoUs.append(IoU)
        
        Dice = (2 * intersection) / (pred_cls.sum() + gt_cls.sum()) if (pred_cls.sum() + gt_cls.sum()) != 0 else 0
        Dice_scores.append(Dice)
        
        TP = intersection
        FP = pred_cls.sum() - TP
        FN = gt_cls.sum() - TP
        
        Precision = TP / (TP + FP) if (TP + FP) != 0 else 0
        Recall = TP / (TP + FN) if (TP + FN) != 0 else 0
        F1 = 2 * (Precision * Recall) / (Precision + Recall) if (Precision + Recall) != 0 else 0
        
        Precisions.append(Precision)
        Recalls.append(Recall)
        F1_scores.append(F1)
    
    mIoU = np.mean(IoUs)
    Dice = np.mean(Dice_scores)
    Precision = np.mean(Precisions)
    Recall = np.mean(Recalls)
    F1 = np.mean(F1_scores)
    
    return mIoU, Dice, Precision, Recall, F1


## Fuzzy logic and BP



In [8]:
#fuzzy logic
#MF
#MF1 for r   -30~-10 sigmoid 
def MF1(x):
    if x< -30:
        return 0
    if x> -10:
        return 1
    return 1/(1+np.exp(-0.1*(x+20)))
#MF2 for w  0.2~0.5 sigmoid
def MF2(x):
    if x< 0.2:
        return 0
    if x> 0.5:
        return 1
    return 1/(1+np.exp(-10*(x-0.35)))

#MF3 for v -0.23~0.23 
def MF3(x):
    if x< -0.23:
        return 0
    if x> 0.23:
        return 0
    return 1-abs(x)/0.23

#MF4 for text_r 0~30 
def MF4(x):
    if x< 0:
        return 0
    if x> 30:
        return 0
    if x<=5:
        return x/5
    else:
        return 1-(x-5)/25 
#fuzzy logic   
def fuzzy_logic(r, v, w, text_r):
    fuzzy_result = np.sum([MF1(r), MF2(w), MF3(v), MF4(text_r)]) / 4
    if fuzzy_result < 0.3:
        return 1
    return 2

In [2]:
#BP
# load the model
from joblib import dump, load
bp = load('model_BP.joblib')

d:\Program\Anaconda\envs\torch22\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelBinarizer from version 1.2.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
d:\Program\Anaconda\envs\torch22\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MLPClassifier from version 1.2.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [4]:


from utils.data_vis import plot_img_and_mask, plot_imgs, plot_mask
import os
import numpy as np
import matplotlib.pyplot as plt
from utils.data_vis import plot_img_and_mask, plot_imgs, plot_mask,  Visualize_type,Visualize
import time  # 导入 time 模块

imgs_path = r'F:\Workspace\Projects\气象局技能大赛\基于机器学习的晴空回波识别\Data_test\imgs'
masks_path = r'F:\Workspace\Projects\气象局技能大赛\基于机器学习的晴空回波识别\Data_test\masks'

files_sv = os.listdir(masks_path)
files_sv = [f.split('.')[0] for f in files_sv]

ground_truth = []
predict_mask_bp = []
predict_mask_fuzzy = []
total_time_bp = 0
total_time_fuzzy = 0

for f in files_sv:
    img = np.load(os.path.join(imgs_path, f + '.npy'))
    r = img[:, :, 0]
    v = img[:, :, 1]
    w = img[:, :, 2]
    ldr = img[:, :, 3]
    mask = np.load(os.path.join(masks_path, f + '.npy'))
    echo_mask = np.full(mask.shape, np.nan)
    echo_mask[r >= -50] = 1  # Clear-air echo
    echo_mask[v >= -15] = 1   # Clear-air echo
    echo_mask[w >= 0] = 1     # Clear-air echo
    echo_mask[mask == 1] += 1  # Meteorological echo
    echo_mask[:, 150:][echo_mask[:, 150:] == 1] = 2  #Meteorological echo
    mask[echo_mask == 2] = 1  # Adjust mask for Meteorological echo

    #plot_imgs(img)
    echomask=echo_mask
    echomask [np.isnan(echomask)] = 0

    start_time_dl = time.time()
    echo=echomask.copy()
    echo[echo>=1]=1

    end_time_dl = time.time()

    start_time_feature = time.time()
    # Calculate the texture feature
    Text_r_m = np.zeros_like(r)
    Text_v_m = np.zeros_like(v)
    Text_w_m = np.zeros_like(w)
    Height = np.arange(r.shape[1]) * 0.03  # Height based on range
    for i in range(1, r.shape[0]):
        for j in range(1, r.shape[1]):
            Text_r_m[i, j] = np.abs(r[i, j] - r[i - 1, j])**2
            Text_v_m[i, j] = np.abs(v[i, j] - v[i - 1, j])**2
            Text_w_m[i, j] = np.abs(w[i, j] - w[i - 1, j])**2
            

    Text_r = np.zeros_like(r)
    Text_v = np.zeros_like(v)
    Text_w = np.zeros_like(w)
    H= np.zeros_like(r)
    
    for i in range(r.shape[0]):
        for j in range(r.shape[1]):
            i_min = max(i - 2, 0)
            i_max = min(i + 2, r.shape[0] - 1)
            j_min = max(j - 2, 0)
            j_max = min(j + 2, r.shape[1] - 1)
            
            Text_r[i, j] = np.sum(Text_r_m[i_min:i_max + 1, j_min:j_max + 1]) / ((i_max - i_min + 1) * (j_max - j_min + 1))
            Text_v[i, j] = np.sum(Text_v_m[i_min:i_max + 1, j_min:j_max + 1]) / ((i_max - i_min + 1) * (j_max - j_min + 1))
            Text_w[i, j] = np.sum(Text_w_m[i_min:i_max + 1, j_min:j_max + 1]) / ((i_max - i_min + 1) * (j_max - j_min + 1))
            H[i, j] = Height[j]
    end_time_feature = time.time()
    total_time_bp += end_time_feature - start_time_feature
    total_time_fuzzy += end_time_feature - start_time_feature

    echomask_pred_bp = np.zeros_like(echomask)
    echomask_pred_fuzzy = np.zeros_like(echomask)
    

    start_time_bp = time.time()
    for i in range(2, r.shape[0] - 2):
        for j in range(2, r.shape[1] - 2):
            r_ = r[i, j]
            v_ = v[i, j]
            w_ = w[i, j]
            ldr_ = ldr[i, j]
            text_r_ = Text_r[i, j]
            text_v_ = Text_v[i, j]
            text_w_= Text_w[i, j]
            h = H[i, j]
            data = np.array([r_, v_, w_, ldr_, text_r_, text_v_,text_w_, h]).reshape(1, 8)
            data[np.isnan(data)] = -999
            if r_ < -50:
                echomask_pred_bp[i, j] = 0
                
            else:
                echomask_pred_bp[i, j] = bp.predict(data)[0]
    end_time_bp = time.time()
    total_time_bp += end_time_bp - start_time_bp

    start_time_fuzzy = time.time()
    for i in range(2, r.shape[0] - 2):
        for j in range(2, r.shape[1] - 2):
            r_ = r[i, j]
            v_ = v[i, j]
            w_ = w[i, j]
            ldr_ = ldr[i, j]
            text_r_ = Text_r[i, j]
            data[np.isnan(data)] = -999
            if r_ < -50:
                echomask_pred_fuzzy[i, j] = 0
            else:
                echomask_pred_fuzzy[i, j] = fuzzy_logic(r_, v_, w_, text_r_)
    end_time_fuzzy = time.time()
    total_time_fuzzy += end_time_fuzzy - start_time_fuzzy

    Height=np.array(range(len(r[0])))*0.03
    Time=np.array(range(len(r)))
    echomask_pred_bp[:,150:][echomask_pred_bp[:,150:]==1]=2
    echomask_pred_fuzzy[:,150:][echomask_pred_fuzzy[:,150:]==1]=2
    ground_truth.append(echomask)
    predict_mask_bp.append(echomask_pred_bp)
    predict_mask_fuzzy.append(echomask_pred_fuzzy)



In [9]:
#flatten
predict_mask_bp_list = []
predict_mask_fuzzy_list = []
ground_truth_list = []
for i in range(len(predict_mask_bp)):
    predict_mask_bp_list.extend(predict_mask_bp[i].flatten())
    predict_mask_fuzzy_list.extend(predict_mask_fuzzy[i].flatten())
    ground_truth_list.extend(ground_truth[i].flatten())
#to np.array
predict_mask_bp_list = np.array(predict_mask_bp_list)
predict_mask_fuzzy_list = np.array(predict_mask_fuzzy_list)
ground_truth_list = np.array(ground_truth_list)
'''predict_mask_bp_list[predict_mask_bp_list==2]=0
predict_mask_fuzzy_list[predict_mask_fuzzy_list==2]=0
ground_truth_list[ground_truth_list==2]=0'''

print('Total time for BP: ', total_time_bp)
print('Total time for fuzzy logic: ', total_time_fuzzy)
#evaluation
mIoU_bp, Dice_bp, Precision_bp, Recall_bp, F1_bp = evaluation(predict_mask_bp_list, ground_truth_list)
mIoU_fuzzy, Dice_fuzzy, Precision_fuzzy, Recall_fuzzy, F1_fuzzy = evaluation(predict_mask_fuzzy_list, ground_truth_list)

print('BP: mIoU: ', mIoU_bp, 'Dice: ', Dice_bp, 'Precision: ', Precision_bp, 'Recall: ', Recall_bp, 'F1: ', F1_bp)
print('Fuzzy logic: mIoU: ', mIoU_fuzzy, 'Dice: ', Dice_fuzzy, 'Precision: ', Precision_fuzzy, 'Recall: ', Recall_fuzzy, 'F1: ', F1_fuzzy)

Total time for BP:  215.69961500167847
Total time for fuzzy logic:  115.10761737823486
BP: mIoU:  0.7617916755637565 Dice:  0.8322362421390457 Precision:  0.9037146392112914 Recall:  0.7982138052202433 F1:  0.8322362421390457
Fuzzy logic: mIoU:  0.7411400983657757 Dice:  0.821116688703604 Precision:  0.800521367199853 Recall:  0.8666665088609099 F1:  0.8211166887036039


## Deep Learning

In [26]:
import argparse
import logging
import os
import numpy as np
import torch
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms
from unet import UNet, UNet2Plus, UNet3Plus, TransUNet, UNetWithSE
from utils.dataset import BasicDataset
from utils.eval import calculate_iou
from utils.data_vis import plot_img_and_mask, plot_imgs, plot_mask, Visualize_type


def predict_img(unet_type, net, full_img, device, scale_factor=1, out_threshold=0.5):
    """
    Generate a predicted mask for the given image using a U-Net model.
    Args:
    - unet_type (str): Type of U-Net model.
    - net (torch.nn.Module): The trained U-Net model.
    - full_img (np.ndarray): Input image (H, W, C).
    - device (torch.device): Computational device (CPU or GPU).
    - scale_factor (float): Scale factor for the input image.
    - out_threshold (float): Minimum probability threshold for the mask.
    
    Returns:
    - np.ndarray: Binary mask after thresholding.
    """
    net.eval()  # Set model to evaluation mode
    img = np.array(full_img, dtype=np.float32)

    # Handle NaN and inf values
    img[np.isnan(img)] = -99
    img[np.isinf(img)] = -99

    # Preprocess the image
    img = torch.from_numpy(BasicDataset.preprocess(unet_type, img, scale_factor))
    img = img.unsqueeze(0).to(device=device, dtype=torch.float32)

    with torch.no_grad():
        output = net(img)
        probs = torch.sigmoid(output) if net.n_classes == 1 else F.softmax(output, dim=1)

        # Convert prediction to PIL image and resize to original size
        probs = probs.squeeze(0)
        tf = transforms.Compose([transforms.ToPILImage(), transforms.Resize(full_img.shape[1]), transforms.ToTensor()])
        probs = tf(probs.cpu())

        # Return thresholded mask
        return probs.squeeze().cpu().numpy() > out_threshold


def slide_window_predict(unet_type, net, full_img, device, scale_factor=1, out_threshold=0.5):
    """
    Predict mask for large images using sliding window approach. Divides the image into smaller blocks.
    Args:
    - unet_type (str): Type of U-Net model.
    - net (torch.nn.Module): Trained U-Net model.
    - full_img (np.ndarray): Input image (H, W, C).
    - device (torch.device): Computational device (CPU or GPU).
    - scale_factor (float): Scale factor for the input image.
    - out_threshold (float): Minimum probability threshold for the mask.
    
    Returns:
    - np.ndarray: Binary mask for the entire image.
    """
    windows_size = 400
    stride = 80
    num_w = (full_img.shape[0] - windows_size) // stride + 1
    num_h = 1  # For simplicity, using 1 horizontal window, adjust as needed

    count_predict = np.zeros((full_img.shape[0], full_img.shape[1]))
    count_overlap = np.zeros((full_img.shape[0], full_img.shape[1]))

    for i in range(num_w):
        for j in range(num_h):
            img = full_img[i * stride:i * stride + windows_size, j * stride:j * stride + windows_size, :]
            mask = predict_img(unet_type, net, img, device, scale_factor, out_threshold)

            count_predict[i * stride:i * stride + windows_size, j * stride:j * stride + windows_size] += mask
            count_overlap[i * stride:i * stride + windows_size, j * stride:j * stride + windows_size] += 1

            # Edge handling for the last window
            if i == num_w - 1:
                img = full_img[full_img.shape[0] - windows_size:, j * stride:j * stride + windows_size, :]
                mask = predict_img(unet_type, net, img, device, scale_factor, out_threshold)
                count_predict[full_img.shape[0] - windows_size:, j * stride:j * stride + windows_size] += mask
                count_overlap[full_img.shape[0] - windows_size:, j * stride:j * stride + windows_size] += 1

            if j == num_h - 1 and i == num_w - 1:
                img = full_img[full_img.shape[0] - windows_size:, full_img.shape[1] - windows_size:, :]
                mask = predict_img(unet_type, net, img, device, scale_factor, out_threshold)
                count_predict[full_img.shape[0] - windows_size:, full_img.shape[1] - windows_size:] += mask
                count_overlap[full_img.shape[0] - windows_size:, full_img.shape[1] - windows_size:] += 1

    # Combine predictions
    mask = count_predict / count_overlap
    mask = mask > out_threshold

    # Post-process the mask
    mask[:, 150:][full_img[:, 150:, 0] >= -50] = True
    mask[:, 150:][full_img[:, 150:, 1] >= -15] = True
    mask[:, 150:][full_img[:, 150:, 2] >= 0] = True

    return mask



def predict_met_echo(img,  unet_type, net, device):
        """
        Predict echo mask from radar data (loaded from .npy files) using the trained U-Net model.

        - unet_type (str): Type of U-Net model.
        - net (torch.nn.Module): Trained U-Net model.

        
        Returns:
        - np.ndarray: Predicted mask.
        """


        img = img

        radar_r = img[:, :, 0]
        radar_v = img[:, :, 1]
        radar_w = img[:, :, 2]
        radar_depomask = img[:, :, 3]

        # Predict mask based on image size
        if img.shape == (400, 400, 4):
            mask = predict_img(unet_type, net, img, device)
        else:
            mask = slide_window_predict(unet_type, net, img, device)

        # Post-process the mask
        echo_mask = np.full(mask.shape, 0)
        echo_mask[radar_r >= -50] = 1
        echo_mask[radar_v >= -15] = 1
        echo_mask[radar_w >= 0] = 1
        echo_mask[mask == 1] += 1
        echo_mask[:, 150:][echo_mask[:, 150:] == 1] = 2  # Post-process for higher altitudes


        return echo_mask

## Unet +SE

In [36]:

unet_type='se'
gpu_id=0
if unet_type == 'trans':
        net = TransUNet(img_dim=400, in_channels=4, out_channels=128, head_num=4, mlp_dim=512, block_num=8, patch_dim=16, class_num=1)
        model = r'model\TransUnet\CP_epoch60_miou_0.8984.pth'
elif unet_type == 'se':
        net = UNetWithSE(n_channels=4, n_classes=1)
        model = r'model\UnetSE_LDR_randomDROP\CP_epoch60_miou_0.9799.pth'
else:
        net = UNet(n_channels=4, n_classes=1)
        model = r'model\Unet\CP_epoch1_miou_tensor(0.9825).pth'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net.to(device=device)
net.load_state_dict(torch.load(model, map_location=device))

    

<All keys matched successfully>

In [39]:


from utils.data_vis import plot_img_and_mask, plot_imgs, plot_mask
import os
import numpy as np
import matplotlib.pyplot as plt
from utils.data_vis import plot_img_and_mask, plot_imgs, plot_mask,  Visualize_type,Visualize
import time 

imgs_path = r'F:\Workspace\Projects\气象局技能大赛\基于机器学习的晴空回波识别\Data_test\imgs'
masks_path = r'F:\Workspace\Projects\气象局技能大赛\基于机器学习的晴空回波识别\Data_test\masks'

files_sv = os.listdir(masks_path)
files_sv = [f.split('.')[0] for f in files_sv]

ground_truth = []
predict_mask_dl = []

total_time_dl = 0

for f in files_sv:
    img = np.load(os.path.join(imgs_path, f + '.npy'))
    r = img[:, :, 0]
    v = img[:, :, 1]
    w = img[:, :, 2]
    ldr = img[:, :, 3]
    img[:, :, 3]=np.full_like(ldr,np.nan)
    mask = np.load(os.path.join(masks_path, f + '.npy'))
    r=img[:,:,0]
    v=img[:,:,1]
    w=img[:,:,2]
    Height=np.array(range(len(r[0])))*0.03
    Time=np.array(range(len(r)))
    echomask=np.full(mask.shape,np.nan)
    echomask[r >= -50] = 1
    echomask[v >=-15] = 1
    echomask[w >=0] = 1
    echomask[mask==1]+=1
    echomask[:,150:][echomask[:,150:]==1]=2
    echomask [np.isnan(echomask)] = 0

    start_time_dl = time.time()
    echomask_dl= np.zeros_like(echomask)
    echo=echomask.copy()
    echo[echo>=1]=1
    echomask_pred_dl=predict_met_echo(img,unet_type, net, device)
    '''echo[echomask_pred_dl==1]=2
    echomask_pred_dl=echo'''
    end_time_dl = time.time()
    total_time_dl += end_time_dl - start_time_dl




    echomask_pred_dl[echomask_pred_dl>2]=2

    predict_mask_dl.append(echomask_pred_dl)
    ground_truth.append(echomask)
    
print('Total time for deep learning: ', total_time_dl)



C:\Users\85000\AppData\Local\Temp\ipykernel_36748\2808364713.py:97: RuntimeWarning: invalid value encountered in divide
  mask = count_predict / count_overlap


Total time for deep learning:  5.524238109588623


In [40]:
# Flatten the masks
predict_mask_dl_list = []
ground_truth_list = []
for i in range(len(predict_mask_dl)):
    predict_mask_dl_list.extend(predict_mask_dl[i].flatten())
    ground_truth_list.extend(ground_truth[i].flatten())

# Convert to np.array
predict_mask_dl_list = np.array(predict_mask_dl_list)
ground_truth_list = np.array(ground_truth_list)
'''predict_mask_dl_list[predict_mask_dl_list==2]=0
ground_truth_list[ground_truth_list==2]=0'''
print('Total time for Deep Learning: ', total_time_dl)
# Evaluation
mIoU_dl, Dice_dl, Precision_dl, Recall_dl, F1_dl = evaluation(predict_mask_dl_list, ground_truth_list)
print('Deep Learning: mIoU: ', mIoU_dl, 'Dice: ', Dice_dl, 'Precision: ', Precision_dl, 'Recall: ', Recall_dl, 'F1: ', F1_dl)

Total time for Deep Learning:  5.524238109588623
Deep Learning: mIoU:  0.968783335023918 Dice:  0.9837854293148531 Precision:  0.9853790100683537 Recall:  0.9822129938806277 F1:  0.983785429314853
